<a href="https://colab.research.google.com/github/IkeLyons/Wine-Quality-Neural-Net/blob/main/WineQualityNeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1 (resources used)

https://www.tensorflow.org/api_docs/python/tf/Variable
https://www.tensorflow.org/guide/variable

The tensorflow variable is a crucial resource that I am using from the tensorflow library. They are used to represent the weights and bias layers and can be connected together to form the whole neural network.


https://www.tensorflow.org/api_docs/python/tf/random/truncated_normal

This is the resource I used in order to initialize the weights of my hidden layer. I used this resource because as we learned you cannot initalize the values to 0, and this random inializer is tensorflows recommended initializer method for Neural Netowrk weights. 

# Data seleection and importing

In this project we are going to attempt to predict the quality of white wines on a scale of 1 to 10. The dataset includes a number of chemical indicators that might be useful in determining the quality of a white wine.

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download brendan45774/wine-quality

wine-quality.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
#! unzip wine-quality.zip

Archive:  wine-quality.zip
replace winequality-red.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace winequality-white.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [6]:
import pandas as pd
dataset = pd.read_csv('winequality-white.csv', encoding='utf-8', sep=";")
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


# Exploratory Data Analysis

In [7]:
dataset.shape

(4898, 12)

In [8]:
dataset.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
dataset.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


# Train-Dev-Test Split

In [10]:
X = dataset.drop('quality', 1)
y = dataset['quality']

In [11]:
from sklearn.model_selection import train_test_split

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

# Forward Propagation Implementation

In [12]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [19]:
import tensorflow as tf

hiddenLayerSize = 20
num_features = X_train.shape[1]
print(num_features)

W1 = tf.Variable(tf.truncated_normal([hiddenLayerSize, num_features], dtype=tf.float32, name='Weights1', stddev=0.1))
b1 = tf.Variable(tf.zeros([hiddenLayerSize], tf.float32), name='bias1')

11
